En este tutorial se abordan como hacer uso basico de un LLM usando LangChain, como usar prompt templates y pasarle mensajes al modelo.

# Usando Modelos de Lenguaje

LangChain soporta diferentes LLMs y se pueden intercambiar entre ellos, pero hay que considerar que no todos son 100% compatibles entre sí.

In [11]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

ChatModels son instancias "Runnables" de LangChain, lo que significa que tienen una inferzas estandar para interactuar y permiten que estos sean:
* Invocados (invoked): Una sola input es transformada en output.
* Batched: Multiples inputs pueden ser transformados eficentemente en outputs.
* Streamed: Las outputs son streameadas así como vayan siendo producidas.
* Inspeccionada: Información del esquema del input, output, y configuración del Runnable pueden ser accedidos.
* Composed: Multiples Runnables pueden ser moldeados para trabajar juntos para crear pipelines.

A continuación de usa el método .invoke para procesar como input a messages, que consiste de una lista de dos funciones que definen un mensaje al modelo.

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English to Chinese"),
    HumanMessage("how are you?")
]

model.invoke(messages)

AIMessage(content='你好吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 22, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-f396eae2-4dfb-412a-bd06-06ea58c7023e-0', usage_metadata={'input_tokens': 22, 'output_tokens': 4, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

ChatModels recibe un objeto de mensaje como entrada y genera un objeto de mensaje como salida. Además del contenido en texto, el objeto de mensaje contiene información adicional como roles en la conversación, llamadas a herramientas y detalles del uso de tokens.

LangChain tambien soporta entradas para el modelo con strings o formatos de OpenAI:

In [4]:
model.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd9b8d9a-b3ea-424d-b612-7345f859c07c-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
model.invoke([{"role":"user","content":"Hello"}])

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-a06a170f-f247-4c89-bcc6-c167089b53b1-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
model.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 8, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-90fc717e-cc62-43bb-967c-7c40d770b59e-0', usage_metadata={'input_tokens': 8, 'output_tokens': 10, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Streaming

Para una mejor experiencia de usuario, se puede leer la respuesta del modelo mientras se esta generando en tiempo real (streaming), el ChatModels permite tomar uso del streaming para leer los tokens mientras se van generando.

Importante considerar que no todos los modelos soportan streamming.

In [12]:
messages = [
    SystemMessage("Translate the following from English to Chinese"),
    HumanMessage("how are you? My name is Dexter, I am a data engineer from UPY. I was born on 2002. I have experience building data pipelines on Azure environments")
]

for token in model.stream(messages):
    print(token.content, end="|")

|你|好吗|？|我|叫|德|克|斯|特|，我|是| UP|Y| 的|数据|工程|师|。我|出生|于| |200|2| 年|。我|在| Azure| 环|境|中|构|建|数据|管|道|方面|有|经验|。||

# Prompt Templates

En las ejecuciones anteriores se han pasando las prompts al modelo como listas con la información necesaria para lograr una tarea.

Las Prompt Templates son herramientas de LangChain que facilitan la estructuración de una prompt para evitar estar re-escribiendo listas con la prompt adecuada. Estas Templates toman el input del usuario y devuelven una prompt con una estructura previamente definida lista para enviarse al modelo.

El siguiente Prompt Template toma del usuario dos vairables:
* laguage : El lenguaje al que se va a traducir el texto.
* text : El texto a traducir

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system",system_template),("user","{text}")]
)

ChatPromptTemplate soporta diferentes roles de mensaje en un mismo Prompt Template. language es el parametro del mensaje de sistema, y el text del usuario es parte del mensaje del usuario.

In [16]:
prompt = prompt_template.invoke({"language":"Italian","text":"Hi. How are you?"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi. How are you?', additional_kwargs={}, response_metadata={})])

Retorna un ChatPromptValue con dos mensajes. Si se quiere acceder solamente a los mensajes, se puede usar la siguiente funcion:

In [17]:
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi. How are you?', additional_kwargs={}, response_metadata={})]

In [18]:
response = model.invoke(prompt)
print(response.content)

Ciao. Come stai?


In [19]:
response

AIMessage(content='Ciao. Come stai?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 24, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-88e1bba5-00df-4a60-a2c9-215bcef10683-0', usage_metadata={'input_tokens': 24, 'output_tokens': 8, 'total_tokens': 32, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})